<a href="https://colab.research.google.com/github/MrYousaf128/Project/blob/main/FAKE_NEWS_DETECTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("clmentbisaillon/fake-and-real-news-dataset")

print("Path to dataset files:", path)

100%|██████████| 41.0M/41.0M [00:00<00:00, 150MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/clmentbisaillon/fake-and-real-news-dataset/versions/1


In [ ]:
import pandas as pd
True_df = pd.read_csv(path + "/True.csv")
Fake_df = pd.read_csv(path + "/Fake.csv")
True_df['label'] = 1
Fake_df['label'] = 0
dataset = pd.concat([True_df, Fake_df])




,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [ ]:
dataset = dataset.sample(frac=1).reset_index(drop=True)

In [ ]:

dataset.drop(columns=['title','subject','date'],inplace=True)

In [ ]:
dataset.head()

,text,label
0,PHNOM PENH (Reuters) - Cambodia has canceled t...,1
1,Cassandra s points about why she s supporting ...,0
2,Joyce Curnell s family has been destroyed by S...,0
3,MOSCOW (Reuters) - Russia said it will retalia...,1
4,WASHINGTON (Reuters) - The U.S. Senate on Frid...,1


In [ ]:
import string
import re
def wordopt(text):
  text = text.lower()
  text = re.sub("\[.*?/]", '')
  text = re.sub("\\W"," ",text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  return text

In [ ]:
dataset['text'] = dataset['text'].apply(wordopt)

In [ ]:
X = dataset['text']
Y = dataset['label']

In [ ]:
from sklearn.model_selection import train_test_split
X_train ,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.25)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(X_train)
xv_test = vectorization.transform(X_test)

# Save the vectorizer
joblib.dump(vectorization, 'tfidf_vectorizer.pkl')

['tfidf_vectorizer.pkl']

In [ ]:
from sklearn.linear_model import LogisticRegression
import joblib

LR = LogisticRegression()
LR.fit(xv_train,Y_train)

# Save the model
joblib.dump(LR, 'logistic_regression_model.pkl')

['logistic_regression_model.pkl']

In [ ]:
predictt = LR.predict(xv_test)

In [ ]:
LR.score(xv_test, Y_test)

0.9860133630289533

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(Y_test, predictt))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5831
           1       0.98      0.99      0.99      5394

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test, predictt)


array([[5745,   86],
       [  71, 5323]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, predictt)

0.9860133630289533

In [ ]:
!pip install Gradio
import gradio as gr
import joblib
import string
import re


# ----------------------------
# Load Model and Vectorizer
# ----------------------------
try:
    vectorizer = joblib.load("tfidf_vectorizer.pkl")
    model = joblib.load("logistic_regression_model.pkl")
except FileNotFoundError:
    raise FileNotFoundError("Please run the cells to train and save the model and vectorizer first.")
# ----------------------------
# Text Cleaning Function
# ----------------------------
def wordopt(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# ----------------------------
# Prediction Function
# ----------------------------
def predict_news(news_text):
    if not news_text.strip():
        return "⚠️ Please enter some news text."

    cleaned = wordopt(news_text)
    vectorized = vectorizer.transform([cleaned])
    prediction = model.predict(vectorized)

    if prediction[0] == 0:
        return "🚨 This news is likely FAKE."
    else:
        return "✅ This news is likely REAL."

# ----------------------------
# Launch Gradio App
# ----------------------------
gr.Interface(
    fn=predict_news,
    inputs=gr.Textbox(lines=8, placeholder="Enter news text here..."),
    outputs=gr.Text(label="Prediction"),
    title="📰 Fake News Classifier",
    description="Enter a news article or paragraph to check if it's likely FAKE or REAL."
).launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ec9f6d9e120c48b959.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
